In [33]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path
from tqdm.auto import tqdm  

from gym_trading_env.downloader import download
import datetime

c:\Users\cleme\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
download_dir = "data/raw"
processed_dir = "data/processed/"

TARGET_TIMEFRAME = pd.Timedelta("30m")
download_timeframe = "5m"

# Download

In [ ]:
download(
    exchange_names = ["binance", "bitfinex2", "huobi"],
    symbols= ["BTC/USDT", "ETH/USDT"],
    timeframe= download_timeframe,
    dir = download_dir,
    since= datetime.datetime(year= 2019, month= 1, day=1),
)

# Process

In [37]:
pathes = glob.glob(f"{download_dir}/*pkl")
for path in tqdm(pathes):
    name = Path(path).name.split(".")[0]
    df  = pd.read_pickle(path)
    timeframe = (df.index - df.index.to_series().shift(1)).value_counts().index[0]
    for offset in range(TARGET_TIMEFRAME//timeframe):
        process_df = df.resample("30min", offset= offset*timeframe).agg({
            "date_close": lambda x : x[-1] if len(x) > 0 else np.nan,
            "open": lambda x : x[0] if len(x) > 0 else np.nan,
            "high": lambda x : max(x) if len(x) > 0 else np.nan,
            "low": lambda x : min(x) if len(x) > 0 else np.nan,
            "close": lambda x : x[-1] if len(x) > 0 else np.nan,
            "volume": lambda x : sum(x) if len(x) > 0 else np.nan
        })[1:-1]
        process_df.dropna(inplace = True)
        process_df.to_pickle(f"data/processed/{name}-{offset}.pkl")

100%|██████████| 6/6 [12:36<00:00, 126.04s/it]
